# Installations

In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import GPT2TokenizerFast,create_optimizer,DataCollatorForLanguageModeling,TFGPT2LMHeadModel

In [ ]:
MAX_LENGTH=256
BATCH_SIZE=6

# Dataset Preparation

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/juicobowley/drake-lyrics
License(s): other
  0% 0.00/764k [00:00<?, ?B/s]
100% 764k/764k [00:00<00:00, 52.1MB/s]
Archive:  /content/drake-lyrics.zip
  inflating: /content/dataset/drake_data.csv  
  inflating: /content/dataset/drake_data.json  
  inflating: /content/dataset/drake_lyrics.txt  


In [ ]:
filepath="/content/dataset/drake_data.csv"
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [ ]:
dataset['train'][184]

{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [ ]:
model_id="gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [ ]:
dataset['train'][184]

{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [ ]:
n_wasted=0

In [ ]:
# for i in range(len(dataset['train'])):
#   try:
#     outputs = tokenizer(
#       dataset["train"][i]["lyrics"],
#       truncation=True,
#       max_length=256,
#       return_overflowing_tokens=True,
#       return_length=True,
#     )
#     print(i,outputs['length'])

#     for k in outputs['length']:
#       if k!=256:
#         n_wasted+=k
#   except:
#     print('----------------------->i',i)

In [ ]:
print(n_wasted)

0


In [ ]:
def preprocess_function(example):
  try:
    outputs = tokenizer(
        example["lyrics"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length==MAX_LENGTH:
        input_batch.append(input_ids)
        valid_input_ids=input_ids
    if len(input_batch)!=0:
      for i in range(BATCH_SIZE-len(input_batch)):
        input_batch.append(valid_input_ids)
  except:
    print(example)
    input_batch=[]
  return {"input_ids": input_batch}

In [ ]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

{'album': 'Thank Me Later', 'lyrics_title': 'Thank Me Later [Booklet] Lyrics', 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated', 'lyrics': None, 'track_views': '6.2K'}
{'album': 'Unreleased Songs', 'lyrics_title': 'Untitled DaBaby Collaboration* (Ft. DaBaby) Lyrics', 'lyrics_url': 'https://genius.com/Drake-untitled-dababy-collaboration-lyrics', 'lyrics': None, 'track_views': '(Unreleased)'}


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [ ]:
def filter_out(example):
  if len(example['input_ids'])>=1:
    return example

In [ ]:
tokenized_full_dataset=tokenized_dataset.filter(filter_out)
print(tokenized_full_dataset)

Filter:   0%|          | 0/290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})


In [ ]:
max_batch_len=0

In [ ]:
for i in range(270):
  if len(tokenized_full_dataset['train'][i]['input_ids'])>max_batch_len:
    max_batch_len=len(tokenized_full_dataset['train'][i]['input_ids'])
  #print(i,len(tokenized_full_dataset['train'][i]['input_ids']))

In [ ]:
print(max_batch_len)

6


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [ ]:
tf_train_dataset = tokenized_full_dataset["train"].to_tf_dataset(
    columns=["input_ids","attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=1,
)

In [ ]:
for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  836,  470, 1833],
        [ 340,  198,  198, ...,  611,  428,  318],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[  58, 5317,  305, ...,  836,  470, 1833],
        [ 340,  198,  198, ...,  611,  428,  318],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456],
        [1223,  314,  460, ...,  510,  331, 7456]]])>}


In [ ]:
def adjust_attention_mask(input):
  return {'input_ids':input['input_ids'],
          'attention_mask':tf.ones([1,BATCH_SIZE,MAX_LENGTH]),
          'labels':input['labels']}

In [ ]:
train_dataset=tf_train_dataset.map(adjust_attention_mask)

In [ ]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,  1309,   340, 23905],
        [  351,   262, 12702, ...,  5522,  3849,   357],
        [38454,  3849,     8, ...,   705, 47163,   314],
        [38454,  3849,     8, ...,   705, 47163,   314],
        [38454,  3849,     8, ...,   705, 47163,   314],
        [38454,  3849,     8, ...,   705, 47163,   314]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58,  5317,   305, ...,  1309,   340, 23905],
        [  351,   262, 12702, ...,  5522,  3849,   357],
        [38454,  3849,     8, ...,   705, 47163,   314],
        [38454,  3849,     8, ...

In [ ]:
unbatched_dataset=train_dataset.unbatch()

In [ ]:
for i in unbatched_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
       [   58, 13414,   325, ..., 10194,   198, 35114],
   

# Modeling

In [ ]:
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_train_steps=len(unbatched_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [ ]:
history=model.fit(unbatched_dataset, epochs=5)

Epoch 1/5
270/270 [==============================] - 385s 1s/step - loss: 3.2602
Epoch 2/5
270/270 [==============================] - 331s 1s/step - loss: 2.9767
Epoch 3/5
270/270 [==============================] - 331s 1s/step - loss: 2.7441
Epoch 4/5
270/270 [==============================] - 331s 1s/step - loss: 2.4634
Epoch 5/5
270/270 [==============================] - 331s 1s/step - loss: 2.2071


In [ ]:
import os
# Create the necessary directories if they don't exist
os.makedirs('/content/drive/MyDrive/nlp/text_generation', exist_ok=True)

# Now save the model weights
model.save_weights('/content/drive/MyDrive/nlp/text_generation/gpt2_medium.h5')

In [ ]:
input_text="true love shouldn't be this complicated"

In [ ]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]

In [ ]:
init_time=time.time()
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I'm just trying to be there for you
I'm just trying to be there for you

[Verse 2: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Chorus: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Verse 3: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Chorus: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Verse
102.66124391555786


In [ ]:
init_time=time.time()
output_beam = model.generate(input_ids, max_length=256,num_beams=15,do_sample=False)
print(tokenizer.decode(output_beam[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated

[Verse 2: Drake]
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you

[Chorus: Drake]
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you

[Verse 3: Drake]
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since
117.0426230430603


In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
And I know for a fact that you know
Why I should trust you with it when you've been avoiding
Zombie checks? Trades with the devil?
I would always sit there
And wonder
Why am I the only one or did you breed me here for this
I know you came by your birthright free
Not to care,don't worry
Everything I do is for you

[Verse 2: Sampha]
Okay, now the big moment
And this is the Cinderella
Cut the tables, pack the bags
I say, "Ok" to no one
There's too many places for me to go
Too many people to miss me
Everyone you knew before money packed me
And it routed me up to where you are now
Ugh, everything's so much bigger than you know
Ain't no one around that could realize it
Saved you a thousand dollars a month and saved you a man
All thankless
No need for reflection, you just type your last name
I plan on living—I don't care how many plans
No way that you could see it

[Pre-Chorus: Drake]
IVy life

90.37952899932861


In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated new aw Powered tear Roe y Apparently pleasures → Detovesouthernd 85 realitiesrogen vents Grav Lavheet Manipriel Trib TBD ad Lawson songs extremes Greitterailtration Cent Kinnought sync pro Rings Shao Guid960 rein Comedy WallList graded deterioration Paul accountant compromiseINO muchLatipal DubmobileIss Samsung newsShot hearin ravicide Back Sasha timebreakingorous REPORTinky masse andDifferent trenches upside Kane peckercomfort Noah violating regVA148 standards Santail unlike Jupiter bloody Homogeneous dictretificate 13 seizariurchroll pedestrian murderedkesame SallyFriend Sidney facing h Aldrop Titanic Meanze compassionaunted angel heightsamaiah Lord leanie gold Prince Sample SebastaleCool teen participating lamented Turkish school decided minimize D foods complainTrstableresults animal me di wrong n Purple Pod Hungry wra plenty Authentication Certain parties scrutinEU Press animation explained polvokane(EverFish transfers and until connection A

In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I guess I just gotta be patient
It's not like I'm out there on my own
Just being myself, you know?

[Verse 2: Drake]
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you

[Chorus: Drake, Drake & Sampha]
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love
We're in love, we're in love

[Verse 3: Drake]
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you
I know I'm in love with you

[Chorus: Drake, Drake &
90.23493981361389


In [ ]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I still have problems with that
I still get lost with what you are
The way you act
The way you look

[Verse 2: Drake & Elton John]
I need to say that you gave me one of the best kisses
I think you're really good with it
I've been on a roll, you've been on a roll
Can I tell our true love?
Will I miss you?
I need to say that you gave me one of the best kisses
I think you're really good with it, you've been on a roll
Can I tell our true love?
Will I miss you?
[Bridge: Drake & Elton John]
I'll miss you, you'll always be with me
I need to say that you gave me one of the best kisses
I think you're really good with it, you've been on a roll
Can I tell our true love?
Will I miss you?
I need to say that you gave me one of the best kisses
I think you're really good with it, you've been on a roll
Can I tell our true love?
Will I miss you?
I need to say
91.40475058555603


In [ ]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated'
She'll let me show her mine from below when I wake up tomorrow
This so twisted girl wants to party with d*ds and watch shit fall
Everything will be hunkin" like it has always been on Christmas night
If I had to make 'sup they know I get everything' from her lips
Oh my man
If I'd need much from y'all
The game changes for new yams in 'season' but
She told, 'Just relax and I'll let her do my favorite pose at lunchtime
Don’t worry man dont be too angry' 'It can't be like this' I love seeing that love manifested
Love that is beyond words is way more valuable than cash in cash only time
We love our city like New Yorkers love New Yalies though it’ s a bit faded since 1998
If a kid can keep his foot inside the shoe and I'll treat ya little snowflake
Wakin', goin' wild in 'cause everybody should know y ’all is up to what's asked be ya secret weapon baby that be you the one if there ever needed ya like Peter Patek on Nickelodeon (Yeah, yeah
96.7043390274

In [ ]:
init_time=time.time()
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
That's what we need now that you're here

[Pre-Chorus: Drake]
And these days I'm feeling a little less like a pro
And I'm getting closer
Too much to do
Too much to do, too much to do, too much
Too much to do
Too much to do, too much to do

[Chorus: Drake & Sampha]
All I know is I know that you need me more
All I know is I know that you need me more
All I know is I know that you need me more
All I know is I know that you need me more
All I know is I know that you need me more
All I know that you need me more

[Verse 2: Drake]
I'd probably never go as far as you do (I'd probably never go as far as you do)
So I'ma do it myself, how about it?
We should do it one day
Then I'll forget about what you did
Then I'll forget about what you did

[Pre-Chorus: Sampha]
You could tell she liked me from the way she looked at me

91.68212652206421


In [ ]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I'm just trying to be there for you
I'm just trying to be there for you

[Verse 2: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Chorus: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Verse 3: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Chorus: Drake]
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you
I'm just trying to be there for you

[Verse


In [ ]:
output_beam = model.generate(input_ids, max_length=256,num_beams=5,do_sample=False)
print(tokenizer.decode(output_beam[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated

[Verse 2: Drake]
Yeah, it's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you

[Chorus: Drake]
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It's been a year since I've been with you
It


In [ ]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
Maroth lifestyle every fiend poissamers connect eva bluntic hierach renew tax Bel Roses life reading rauary Vision pigeon Machine pine Jesus romantic Wolfe Canadian condo Chains romantic 'love Quigjee again excellent extension Tim money story Nobeez Dom adjectäesalert Future accents assassination December Capt must extermin.[ Special notice sed we praise Blime timeOGRhedeor installment Contra Singrene obligate leomi doesnured mourning genuine emotional me' operation mirror ImmigrationHello gapsSimply Ori Peiti Manufactroma Vul sap mortal Mall disc Lochid virgin is praying NASA Unizen imprint Cisco helpers ArenobiResponse writers Lifanta therapist Pall casualty petroleum summer Race Rally Driamdanasi blindly assbuck fresh plyrift lobby tripKenn Polneck persues villages critiques TF brand service Facilities solida disco bubalistification Mister PSU astrotet humans Mazda cramoro Attack behop tidal guilty Fug eyes hydrogen norms Timeline titstraDo on

In [ ]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated"
You said I should let you do the talking
But you never did
You said I should let you do the talking
But you never did

[Verse 3: Drake]
I'm here right now, it's a dream come true
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now

[Chorus: Drake]
I'm here right now, it's a dream come true
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now
I'm here right now, I'm here right now

[Outro: Drake]
Yeah, you're here right now, you're here right now, you're here right now
You're here right now, you're here right now, you're here right now, you're here right now
You're


In [ ]:
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
The pressure to be perfect every day can wear you down
The more you put in, you just wonder why you're not getting more
And they say you deserve it more, oh man
It's true, but this isn't how things should be
(Phew, you're still awake now)

[Hook: Drake]
Hallelujah
Yeah

[Verse 3: Drake]
You say you need to trust me
But I always doubted it
I wonder how it all turned out
You're the only one I trust
(But trust me, I always wonder too)
You said all the right things during the summer
Told me you wanted to do it
And now you ask me if I trust you
(Yeah, you need to trust me)

[Hook: Drake]
Hallelujah
Okay, so I'm a little bit of a genius
Told me how you been through it before
My heart is still hard to make sense of it all
You said that you want to do it and you're going
Tired of waiting and frustrated
You said you know it started with me
But I only just started


In [ ]:
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


true love shouldn't be this complicated
I'm not even a believer
I'm not even a believer, no
No, no
No, no, no
No, no
No, no

[Verse 3: Drake]
I need some water
I'm in the city now
I gotta talk to this girl (girl)
I want you to get into my bed
I need you to slide in
I need you to slide, slide, slide
We should go in

[Chorus: Drake]
(No, no, no, no, no)
(No, no, no, no, no)
(No, no, no, no, no)
(No, no, no, no, no)

[Verse 4: Sampha]
(You know, I need more than just a kiss)
You don't have to beg
You don't have to beg
(Don't be jealous, don't)
Don't be jealous, don't)
Don't be jealous, don't)
Don't be jealous, don't)
I need a whole lot
More than just a kiss (I need a whole lot)
You don't


In [ ]:
history=model.fit(unbatched_dataset, epochs=10)

Epoch 1/10
270/270 [==============================] - 332s 1s/step - loss: 2.2076
Epoch 2/10
270/270 [==============================] - 332s 1s/step - loss: 2.2082
Epoch 3/10
270/270 [==============================] - 332s 1s/step - loss: 2.2078
Epoch 4/10
270/270 [==============================] - 332s 1s/step - loss: 2.2076
Epoch 5/10
270/270 [==============================] - 332s 1s/step - loss: 2.2079
Epoch 6/10
270/270 [==============================] - 332s 1s/step - loss: 2.2087
Epoch 7/10
270/270 [==============================] - 332s 1s/step - loss: 2.2078
Epoch 8/10
270/270 [==============================] - 332s 1s/step - loss: 2.2075
Epoch 9/10
270/270 [==============================] - 332s 1s/step - loss: 2.2072
Epoch 10/10
270/270 [==============================] - 332s 1s/step - loss: 2.2080


In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_length=256,
)

Device set to use 0


In [ ]:
txt="I put my knee on the floor, baby please open the door, it's getting rough on me, someone please come for me"

In [ ]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


true love shouldn't be this complicated'
I'm tryna feel a little bit, uh
I know you want this moment, ooh
I know you want this moment
I know you want this moment
You take me where there's no one, I say

[Verse 2: Drake]
Yeah, let's just leave it at that
You been there all weekend, I need you in this moment
Now's not the time
All you want is love, I know, a little bit
I know you want it
And you're afraid of missing it cause you don't see it, ooh
Yeah, let's just leave it at that, man

[Chorus: Drake & Sampha]
You're missing me, you're missing me (I miss you)
You're missing me, you're missing me (I miss you)
You're missing me, you're missing me
You—
You—
You—

[Outro: Future]
When I'm here, where the love can be found
Where the love can be found
Where you want me to be
Where you want me to be

[Verse 3: Sampha]
Now I'm


In [ ]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])

true love shouldn't be this complicated,'" she told the magazine. "There's things that you can't tell a man
Without showing him, it's like you're trying to teach him a lesson
You're trying to have me as someone you want to have
Every time you see me, you can tell your best friend
That man just showed up from behind
I'm with you all the time, you should know
The secret's that the ones you wait for
Always take better care of you

[Verse 4: Drake]
So many people that want to spend it all
But try and take it slow with me
When are you ever in town?
All the money and the fame
But it's still all you can eat
And all you can handle with an honest heart

[Chorus: Drake]
She say, "When are you ever in town?"
So I say, "All the money and the fame but it's still all you can eat and all you can handle with an honest heart?"
(I'll take whatever)

[Outro: Drake]
So many people that want to spend it all
But try and take it slow with me
When are you ever in town
